**Model Training**

In [ ]:
!pip install insightface opencv-python scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 16.3 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1057643 sha256=4be262e1acc6c80333896748b695aa58e5fabb40217f07b285e501da6a246336
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [ ]:
!pip uninstall onnxruntime -y
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from insightface.app import FaceAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import os
import cv2
import numpy as np
import joblib

# Initialize model
model = FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=0)

dataset_path = '/content/drive/MyDrive/small_cv_dataset/CV_DS/CV_DATASET'
X, y = [], []

# Extract embeddings
for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_folder):
        continue
    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        faces = model.get(img)
        if faces:
            # Select the largest face
            faces.sort(key=lambda x: (x.bbox[2]-x.bbox[0]) * (x.bbox[3]-x.bbox[1]), reverse=True)
            X.append(faces[0].embedding)
            y.append(person_name)

X = np.array(X)
y = np.array(y)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train logistic regression classifier
clf = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
clf.fit(X, y_encoded)

# Save model and encoder
joblib.dump(clf, 'face_classifier.joblib')
joblib.dump(le, 'label_encoder.joblib')

print("✅ Training complete with Logistic Regression. Model saved.")


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


**Inference (GUI)**

In [ ]:
!pip install ttkbootstrap opencv-python insightface pillow

In [ ]:
import cv2
import pickle
import numpy as np
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from PIL import Image, ImageTk
from insightface.app import FaceAnalysis
from datetime import datetime
import csv
import os

from joblib import load

recognizer = load(r"C:\Users\Dell\Downloads\cv_project_gui\face_classifier.joblib")
le = load(r"C:\Users\Dell\Downloads\cv_project_gui\label_encoder.joblib")

# Init face recognition model
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

# Initialize attendance log
attendance_log = {}

# Setup GUI
window = ttk.Window(themename="minty")  # Try: cyborg, superhero, flatly
window.title("📋 Face Recognition Attendance System")
window.geometry("1000x750")

# Title
title = ttk.Label(window, text="📷 Live Attendance System", font=("Helvetica", 22, "bold"), bootstyle=PRIMARY)
title.pack(pady=15)

# Video feed display
video_label = ttk.Label(window)
video_label.pack(pady=10)

# Attendance log table
columns = ["Name", "Time"]
table = ttk.Treeview(window, columns=columns, show="headings", height=8, bootstyle="success")
for col in columns:
    table.heading(col, text=col)
    table.column(col, width=200, anchor="center")
table.pack(pady=10)

# Camera
cap = cv2.VideoCapture(0)

def mark_attendance(name):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    if name not in attendance_log:
        attendance_log[name] = now
        table.insert('', 'end', values=(name, now))

def recognize_and_display():
    ret, frame = cap.read()
    if not ret:
        window.after(10, recognize_and_display)
        return

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = app.get(frame_rgb)

    for face in faces:
        bbox = face.bbox.astype(int)
        embedding = face.embedding.reshape(1, -1)

        probs = recognizer.predict_proba(embedding)[0]
        j = np.argmax(probs)
        name = le.classes_[j]
        confidence = probs[j]

        if confidence > 0.90:
            label = f"{name}: {confidence:.2f}"
            mark_attendance(name)
        else:
            label = "Unknown"

        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (70, 255, 70), 2)
        cv2.putText(frame, label, (bbox[0], bbox[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (70, 255, 70), 2)

    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    window.after(10, recognize_and_display)

def export_to_csv():
    filename = f"attendance_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Time"])
        for name, time in attendance_log.items():
            writer.writerow([name, time])
    print(f"[INFO] Attendance exported to {filename}")

def take_screenshot():
    ret, frame = cap.read()
    if ret:
        filename = f"screenshot_{datetime.now().strftime('%H%M%S')}.jpg"
        cv2.imwrite(filename, frame)
        print(f"[INFO] Screenshot saved: {filename}")

def close_app():
    cap.release()
    cv2.destroyAllWindows()
    window.destroy()

# Buttons
btn_frame = ttk.Frame(window)
btn_frame.pack(pady=10)
start_btn = ttk.Button(btn_frame, text="▶ Start", bootstyle=PRIMARY, command=recognize_and_display)
start_btn.grid(row=0, column=0, padx=10)

snap_btn = ttk.Button(btn_frame, text="📸 Screenshot", bootstyle=SUCCESS, command=take_screenshot)
snap_btn.grid(row=0, column=1, padx=10)

export_btn = ttk.Button(btn_frame, text="📁 Export CSV", bootstyle=WARNING, command=export_to_csv)
export_btn.grid(row=0, column=2, padx=10)

exit_btn = ttk.Button(btn_frame, text="❌ Exit", bootstyle=DANGER, command=close_app)
exit_btn.grid(row=0, column=3, padx=10)

window.mainloop()

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
      }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    image_bytes = b64decode(data.split(',')[1])
    img_array = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    cv2.imwrite(filename, img)
    return filename


In [ ]:
from google.colab import files
files.download('face_classifier.joblib')
files.download('label_encoder.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load model + label encoder
clf = joblib.load('face_classifier.joblib')
le = joblib.load('label_encoder.joblib')
model = FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=0)

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o